In [1]:
import csv
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
#prefix 
prefix='NGC4490'

#real photometry first
real_file = "%s.phot" % prefix
real_df = pd.read_csv(real_file, delim_whitespace=True, header=None) #create data frame 
real_df = real_df.iloc[:,:37] #select till relevant end of data frame (quality flag of I)

#name columns
columns=['extension','chip','x','y','chi','snr','sharp','round','maj_ax','crowd','type',
           'cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V','inst_vega_V','mag_V','Verr','chi_V','snr_V',
           'sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I','nrm_ct_rt_I','nrm_ct_rt_err_I', 
           'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I','crowd_I','flag_I']

In [3]:
#assign columns and preview
real_df.columns=columns

#fix y column if not on first chip 
real_df['y'] = real_df.apply(lambda x: x.y+2000 if x.extension>1 else x.y, axis=1)

In [4]:
#establish cuts 
real_cut = real_df[(real_df['crowd_V'] + real_df['crowd_I'] < 0.8) & 
                   ((real_df['sharp_V']+real_df['sharp_I'])**2 <= 0.075) & 
                   (real_df['type'] <= 2) & (real_df['snr_V'] >= 5) & 
                   (real_df['flag_V'] == 0) & (real_df['snr_I'] >= 5) &
                   (real_df['flag_I'] == 0)]

real_cut[:5] #print for a quick look

,extension,chip,x,y,chi,snr,sharp,round,maj_ax,crowd,...,nrm_ct_rt_err_I,inst_vega_I,mag_I,Ierr,chi_I,snr_I,sharp_I,round_I,crowd_I,flag_I
3,1,1,1572.58,2026.43,3.92,496.5,-0.137,0.030,125,0.094,...,9.710000e-11,19.032,99.999,0.004,4.23,251.3,-0.132,0.023,0.095,0
6,1,1,2277.07,970.25,3.78,368.9,-0.084,0.040,70,0.387,...,6.760000e-11,19.592,99.999,0.005,3.64,215.5,-0.079,0.041,0.429,0
9,1,1,1495.72,1658.11,2.85,353.6,-0.088,0.044,55,0.156,...,6.040000e-11,19.742,99.999,0.005,3.03,210.1,-0.089,-0.023,0.166,0
11,1,1,1554.66,1413.73,3.54,321.4,-0.055,0.051,30,0.346,...,5.690000e-11,19.993,99.999,0.006,3.00,176.7,-0.048,0.052,0.351,0
15,1,1,201.60,1285.55,0.80,361.6,-0.020,0.007,45,0.016,...,6.510000e-11,19.254,99.999,0.004,0.82,305.5,-0.011,0.001,0.015,0


In [5]:
#print out real_cut to space separated file
real_cut_file = real_cut.to_csv('%s.phot2' % (prefix), sep=' ', header = True, index = False)

In [6]:
#fake photometry next
fake_file='%s.phot.fake' % prefix #select file 

In [7]:
fake_df = pd.read_csv(fake_file, delim_whitespace=True, header=None) #create data frame 

# calculate number of columns
numberOfColumns = len(fake_df.columns)
numberOfExtraColumns = numberOfColumns - 71
numberOfImagesPerFilter = int(1+(numberOfExtraColumns/30))
numberOfRelevantColumns = (numberOfImagesPerFilter-1)*4

fake_df = fake_df.iloc[:,:(45+(numberOfRelevantColumns))] #select till relevant end of data frame (quality flag of I)

In [8]:
base_columns=['extensionIn','chipIn','xIn','yIn','counts_V1','mag_V1','counts_I1', 'mag_I1','extensionOut','chipOut','xOut','yOut',
           'chi','snr','sharp','round','maj_ax','crowd','type','cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V',
           'inst_vega_V','mag_V','Verr','chi_V','snr_V','sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I',
           'nrm_ct_rt_I','nrm_ct_rt_err_I', 'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I',
           'crowd_I','flag_I'] #base columns

In [9]:
#loop to find correct number of additional photometry columns, and insert them
j = 1
k = 1
l = 1

while (j < numberOfImagesPerFilter):
    m = str(j+1)
    base_columns.insert(k+5,'counts_V%s' % m) 
    base_columns.insert(k+6,'inst_vega_V%s' % m)    
    base_columns.insert(l+9,'counts_I%s' % m) 
    base_columns.insert(l+10,'inst_vega_I%s' % m)    
    j += 1
    k += 2
    l += 4
if j == numberOfImagesPerFilter:
    fake_df.columns=base_columns 
    
# fake_df.columns #print for a quick look

In [10]:
# correct Y position if the star is in the upper CCD
fake_df['yIn'] = fake_df.apply(lambda x: x.yIn+2000 if x.extensionIn>1 else x.yIn, axis=1) 
# fake_df.iloc[:5,:16] #print some relevant columns to see if correct

In [11]:
#make some selection cuts
fake_df['selected']=np.where( ((fake_df['crowd_V'] + fake_df['crowd_I']) < 0.8)
                             &((fake_df['sharp_V']+fake_df['sharp_V'])**2 <= 0.075)
                             & (fake_df['type'] <= 2) 
                             & (fake_df['snr_V'] >= 5) 
                             & (fake_df['flag_V'] == 0) 
                             & (fake_df['snr_I'] >= 5) 
                             & (fake_df['flag_I'] == 0), True, False)

# fake_df['selected'][:10] #print some relevant columns to see if correct

In [12]:
#automation depending on number of filters we have

#initalize columns to store denominator for magnitude conversion
fake_df['vSumCounts'] = 0
fake_df['iSumCounts'] = 0

#find the denominator (sum of counts)
for i in range(numberOfImagesPerFilter):
    vColumnName = 'counts_V%d' %(i+1)
    iColumnName = 'counts_I%d' %(i+1)
    fake_df['vSumCounts'] =  fake_df['vSumCounts'] + fake_df[vColumnName]
    fake_df['iSumCounts'] =  fake_df['iSumCounts'] + fake_df[iColumnName]

In [13]:
#convert to magnitudes
fake_df['dV'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_V/(x.vSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)
fake_df['dI'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_I/(x.iSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)

In [14]:
# fake_df

In [15]:
#copy columns and create file
fake2_df = fake_df[['xIn','yIn','mag_V1','mag_I1','inst_vega_V','inst_vega_I','dV','dI']].copy()
fake2_df[:5] #print to have a quick look
fake_cut_file = fake2_df.to_csv('%s.phot.fake2' % (prefix), sep=' ', header = True, index = False)

In [16]:
fake2_df

,xIn,yIn,mag_V1,mag_I1,inst_vega_V,inst_vega_I,dV,dI
0,46.15,43.72,22.264,22.263,22.275,22.257,0.011,-0.006
1,27.41,47.79,24.426,21.917,24.483,21.932,0.060,0.015
2,37.85,13.89,24.516,23.233,24.489,23.198,-0.024,-0.033
3,0.06,59.12,24.644,22.500,99.999,99.999,99.999,99.999
4,31.81,43.66,24.760,22.790,24.784,22.827,0.024,0.037
5,44.32,56.02,24.957,22.832,25.030,22.830,99.999,99.999
6,22.88,14.90,25.131,24.994,25.127,25.024,99.999,99.999
7,8.44,41.28,25.412,22.816,25.507,22.840,99.999,99.999
8,19.25,20.42,25.996,25.892,25.850,26.038,99.999,99.999
9,50.12,1.83,26.296,25.799,26.516,25.612,99.999,99.999
